In [ ]:
# import sys
# sys.path.insert(0, '/Users/niloufar/Desktop/mlops/mlops')


In [ ]:
pip install tfx

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pandas as pd

import os
import pprint

In [ ]:
# import inspect
# inspect.getfile(tensorflow_model_analysis)

In [ ]:
# pip install ml-metadata
# pip install apache-beam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pp = pprint.PrettyPrinter()

In [35]:
_pipeline_root = './pipeline/'
_data_root = '/content/drive/MyDrive/mlops/course2/week2/data/'
_data_filepath = os.path.join(_data_root, 'adult.csv')

In [36]:
# df = pd.read_csv(_data_filepath, skipinitialspace=True)

In [37]:
# headerList = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']

In [39]:
df = pd.read_csv(_data_root +"adult.csv", skipinitialspace=True)

In [30]:
df.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [31]:
# df.loc[-0.5] = [39, 
#                 'State-gov', 
#                  77516, 
#                 'Bachelors', 
#                  13, 
#                 'Never-married', 
#                 'Adm-clerical', 
#                 'Not-in-family', 
#                 'White', 
#                 'Male', 
#                  2174, 
#                  0, 
#                  40, 
#                 'United-States', 
#                  '<=50K']
# df = df.sort_index().reset_index(drop=True)

In [32]:
# df.to_csv( _data_root +"adult.csv", header=headerList, index=False)
# df.head()

In [40]:
!head {_data_filepath}

age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husban

In [41]:
context = InteractiveContext(pipeline_root=_pipeline_root)

In [42]:
example_gen = CsvExampleGen(input_base=_data_root)

In [43]:
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 3
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [14]:
conn.cursor()